In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from retry import retry

In [32]:
#目黒区
base_url = "https://suumo.jp/chintai/tokyo/sc_meguro/"

In [33]:
@retry(tries=3, delay=10, backoff=2)
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

In [34]:
all_data = []
max_page = 10

In [48]:
for page in range(1, max_page+1):
    url = base_url.format(page)
    soup = get_html(url)
    items = soup.findAll("div", {"class": "cassetteitem"})
    print("page", page, "items", len(items))
    for item in items:
        stations = item.findAll("div", {"class": "cassetteitem_detail-text"})
        for station in stations:
            base_data = {}
            #基礎情報収集
            base_data["名称"] = item.find("div", {"class": "cassetteitem_content-title"}).getText().strip()
            base_data["カテゴリー"] = item.find("div", {"class": "cassetteitem_content-label"}).getText().strip()
            base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).getText().strip()
            base_data["アクセス"] = station.getText().strip()
            base_data["築年数"] = item.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()
            base_data["構造"] = item.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()

            tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")
            for tbody in tbodys:
                data = base_data.copy()
                data["階数"] = tbody.findAll("td")[2].getText().strip()
                data["家賃"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
                data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()
                data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()
                data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()
                data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
                data["面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()
                data["URL"] = "https://suumo.jp" + tbody.findAll("td")[8].find("a").get("href")
                all_data.append(data)

page 1 items 20
page 2 items 20
page 3 items 20
page 4 items 20
page 5 items 20
page 6 items 20
page 7 items 20
page 8 items 20
page 9 items 20
page 10 items 20


In [49]:
df = pd.DataFrame(all_data)
df.to_csv("meguro_raw_data.csv")

In [50]:
df

,名称,カテゴリー,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,URL
0,PASEO目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築4年,4階建,4階,11.9万円,10000円,-,-,ワンルーム,24.4m2,https://suumo.jp/chintai/jnc_000076002819/
1,PASEO目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築4年,4階建,4階,18.3万円,-,-,-,2DK,40.75m2,https://suumo.jp/chintai/jnc_000086717111/
2,PASEO目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築4年,4階建,4階,18.3万円,12000円,-,-,1LDK,40.75m2,https://suumo.jp/chintai/jnc_000085963153/
3,PASEO目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築4年,4階建,1階,19.7万円,12000円,-,-,1SLDK,45.66m2,https://suumo.jp/chintai/jnc_000086509846/
4,PASEO目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築4年,4階建,1階,20.2万円,12000円,-,-,2LDK,45.66m2,https://suumo.jp/chintai/jnc_000076540745/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,トレステージ目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築20年,地下2地上18階建,4階,23万円,-,46万円,23万円,1LDK,55.02m2,https://suumo.jp/chintai/jnc_000083031247/
2461,トレステージ目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築20年,地下2地上18階建,16階,49.8万円,-,99.6万円,49.8万円,4LDK,118m2,https://suumo.jp/chintai/jnc_000086292822/
2462,トレステージ目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築20年,地下2地上18階建,16階,49.8万円,-,99.6万円,49.8万円,4LDK,118m2,https://suumo.jp/chintai/jnc_000086336015/
2463,トレステージ目黒,賃貸マンション,東京都目黒区下目黒２,東急目黒線/武蔵小山駅 歩21分,築20年,地下2地上18階建,16階,49.8万円,-,99.6万円,49.8万円,4LDK,118m2,https://suumo.jp/chintai/jnc_000086298039/


In [52]:

import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'])
# jupyter nbconvert --to script *.ipynb


CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'], returncode=0)